In [62]:
import openpyxl
from openpyxl import load_workbook
from openpyxl.drawing.image import Image
from openpyxl.utils import units
from openpyxl.styles import Border, Side
import requests
from datetime import datetime
import requests
import shutil
import json


In [54]:
def thai_date_format(iso_date_str):
    # แปลง string เป็น datetime object
    dt = datetime.fromisoformat(iso_date_str.replace("Z", "+00:00"))
    
    # สร้างชื่อเดือนภาษาไทย
    months_th = ["", "มกราคม", "กุมภาพันธ์", "มีนาคม", "เมษายน", "พฤษภาคม",
                 "มิถุนายน", "กรกฎาคม", "สิงหาคม", "กันยายน", "ตุลาคม",
                 "พฤศจิกายน", "ธันวาคม"]
    
    day = dt.day
    month = months_th[dt.month]
    year = dt.year + 543  # เปลี่ยนเป็นปี พ.ศ.
    
    return f"{day} {month} {year}"

# ///check Source Form4-1
def check_source_form4_1(ef_source, row, ws_name='ws41'):
    ef_map = {
        'Self collect': 'I',
        'Supplier': 'J',
        'PCR Gen.': 'K',
        'TGO EF': 'L',
        'Int. DB': 'M',
        'Others': 'N'
    }

    if ef_source in ef_map:
        col = ef_map[ef_source]
        cell = f'{col}{row}'
        return f'{ws_name}["{cell}"] = "●"'
    else:
        return f'# ef_source \"{ef_source}\" not recognized'


# กำหนดเส้นกรอบแต่ละด้าน (เส้นบางและเส้นปะ)
thin_border = Border(
    left=Side(border_style="thin", color="000000"),
    right=Side(border_style="thin", color="000000"),
    top=Side(border_style="thin", color="000000"),
    bottom=Side(border_style="thin", color="000000")
)

thin_top = Border(
    top=Side(border_style="thin", color="000000")
)


def set_borderTop(ws, cell_range,position):
    rows = ws[cell_range]
    for row in rows:
        for cell in row:
            if position == 'left':
                cell.border = Border(left=Side(style='dashed'))
            elif position == 'right':
                cell.border = Border(right=Side(style='dashed'))
            elif position == 'top':
                cell.border = Border(top=Side(style='dashed'))
            elif position == 'bottom':
                 cell.border = Border(
                    bottom=Side(style='dashed'),
                    left=Side(style='dashed'),
                    right=Side(style='dashed')
                )
            
def set_borderCenter(ws, cell_range, position):
    rows = ws[cell_range]
    for row in rows:
        for cell in row:
            # อ่าน border เดิมก่อน
            border_old = cell.border

            # กำหนดด้านใหม่ตาม position
            new_border = Border(
                left=border_old.left if position != 'left' else Side(style='mediumDashed'),
                right=border_old.right if position != 'right' else Side(style='mediumDashed'),
                top=border_old.top if position != 'top' else Side(style='mediumDashed'),
                bottom=border_old.bottom if position != 'bottom' else Side(style='mediumDashed')
            )

            cell.border = new_border

def set_border(ws, cell_range):
    rows = ws[cell_range]
    for row in rows:
        for cell in row:
            cell.border = thin_border


In [71]:
form1 = "http://localhost:3000/api/v1/f1/1005/7"
# form4_1= "http://localhost:3000/api/v1/f4-1/form/6"




form1 = requests.get(form1)
if form1.status_code == 200:
    data = form1.json()

    # แสดงข้อมูลจาก JSON
    product = data["product"]
    company = data["company"]
    process = data["process"]

else:
    print("เกิดข้อผิดพลาดในการเรียก API:", response.status_code)
product = data["product"]
company = data["company"]
start = thai_date_format(product["collect_data_start"])
end = thai_date_format(product["collect_data_end"])
date_range = f"{start} - {end}"
submitted_date_thai = thai_date_format(product.get("submitted_date"))
techinfo_list = json.loads(product["product_techinfo"])
product_techinfo_array = [item.strip() for item in techinfo_list]
# form4_1 = requests.get(form4_1)
# if form4_1.status_code == 200:
#     data41 = form4_1.json()

#     form41 = data41["form41"]
#     company = data41["company"]
#     product = data41["product"]
#     process = data41["process"]
#     report41Sum = data41["report41Sum"]

#     print(form41)
# else:
#     print("เกิดข้อผิดพลาดในการเรียก API:", form4_1.status_code)
 

# 1


'Dirt (Max,% wt) = 0.08'

In [ ]:
file_path = "../excel/form_CFP.xlsx"
output_path = "../excel/"+company["name"]+"_"+product["product_name_en"]+".xlsx"
shutil.copy(file_path, output_path)

wb = load_workbook(file_path)
ws01 = wb["Fr-01"] # หรือระบุชื่อ sheet: wb["Sheet1"]

ws01["J2"] = date_range
ws01["F5"] = company.get("name", "")
ws01["F6"] = product.get("product_name_th", "")
ws01["J10"] = product.get("product_name_th", "")
ws01["J11"] = product.get("product_name_en", "")
ws01["J12"] = product.get("scope", "")
ws01["J13"] = product.get("FU_value", "")  + " " +product.get("FU_th", "")
ws01["J14"] = product.get("FU_value", "")  + " " +product.get("FU_en", "")
ws01["J14"] = product.get("PU_value", "")  + " " +product.get("PU_th", "")
ws01["J14"] = product.get("PU_value", "")  + " " +product.get("PU_en", "")
ws01["J17"] = product.get("sale_ratio", "")

start_row = 19
col = "I"
for idx, info in enumerate(product_techinfo_array):
    cell = f"{col}{start_row + idx}"
    ws01[cell] = info

ws01["J24"] = product.get("pcr_reference", "")
ws01["J25"] = submitted_date_thai

# image 
image_url = "../../"+product.get("product_photo")
img = Image(image_url)  
img.width = 300 
img.height = 300  
ws01.add_image(img, "B12")  
######## ---------------------------------------------------------------------
ws02 = wb["Fr-02"]
for i in range(len(process)):
    input_names = []
    row = 13 + (3 * i)
    row_next = row + 1
    
    # ใส่ชื่อ process
    ws02.merge_cells(f"M{row}:O{row_next}")
    ws02[f"M{row}"] = process[i]['process_name']
    set_border(ws02, f"M{row}:O{row_next}")
    
    # ใส่คำว่า "ขนส่ง" และกำหนดเส้นขอบ
    ws02.merge_cells(f"I{row}:I{row_next}")
    ws02[f"I{row}"] = "ขนส่ง"
    set_border(ws02, f"I{row}:I{row_next}")
    ws02[f"H{row_next}"].border = thin_top
    ws02[f"J{row_next}"].border = thin_top
    ws02[f"K{row_next}"].border = thin_top
    
    
    # รวมชื่อ inputs ที่ไม่ใช่ "ผลิตภัณฑ์"
    for j in range(len(process[i]['inputs'])):
        input_names.append(process[i]['inputs'][j]['input_name'])
    input_names_str = ', '.join(input_names)
    
    # ใส่ข้อมูลรวมชื่อ input
    ws02.merge_cells(f"C{row}:F{row_next}")
    ws02[f"C{row}"] = input_names_str
    set_border(ws02, f"C{row}:F{row_next}")

row_end= row_next + 2
# Box Input
ws02.merge_cells(f"B12:B{row_end-1}")
set_borderTop(ws02, f"B12:B{row_end-1}",'left')
ws02.merge_cells(f"G12:G{row_end-1}")
set_borderTop(ws02, f"G12:G{row_end-1}",'right')
ws02.merge_cells(f"B{row_end}:G{row_end}")
set_borderTop(ws02, f"B{row_end}:G{row_end}",'bottom')
# Box Process
ws02.merge_cells(f"L12:L{row_end-1}")
set_borderTop(ws02, f"L12:L{row_end-1}",'left')
ws02.merge_cells(f"P12:P{row_end-1}")
set_borderTop(ws02, f"P12:P{row_end-1}",'right')
ws02.merge_cells(f"L{row_end}:P{row_end}")
set_borderTop(ws02, f"L{row_end}:P{row_end}",'bottom')
# Box Center
# ws02.merge_cells(f"J8:J{row_end+3}")

set_borderCenter(ws02, f"J8:J{row_end+3}",'right')
    
######## ---------------------------------------------------------------------

######## ---------------------------------------------------------------------

# ws41 = wb["Fr-04.1"]
# ws41["B11"] = product.get("product_name_th", "")
# ws41["B12"] = form41.get("item_name", "")
# ws41["C12"] = form41.get("item_unit", "")
# ws41["D12"] = form41.get("item_quantity", "")
# ws41["E12"] = float(form41.get("item_quantity", 0) or 0) * 1.3
# ws41["F12"] = form41.get("lci_source_period", "")
# ws41["G12"] = form41.get("ef", "")
# exec(check_source_form4_1(form41.get("ef_source", ""), 12))
# # ws41["H12"] to ["M12"] form41.get("ef_source", "") 
# # ws41["N12"] = subtitle
# ws41["O12"] = form41.get("ef_source_ref", "")
# ws41["P12"] = form41.get("ratio", "")
# ws41["Q12"] = form41.get("ghg_emission", "")
# ws41["R12"] = form41.get("ghg_emission_proportion", "")
# ws41["S12"] = form41.get("cut_off", "")
# ws41["T12"] = form41.get("description", "")

wb.save(output_path)
print("เขียนข้อมูลใน Excel เรียบร้อยแล้ว")

เขียนข้อมูลใน Excel เรียบร้อยแล้ว
